# LangGraph Example

https://langchain-ai.github.io/langgraph/#example

In [42]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [62]:
@tool
def search(query: str):
    """クエリを使ってWebを検索します"""
    if "東京" in query.lower():
        return "20度で雨が降っています。"
    return "25度で晴れています。"

tools = [search]
tool_node = ToolNode(tools)
tool_node

tools(tags=None, recurse=True, func_accepts_config=True, func_accepts={'writer': False, 'store': True}, tools_by_name={'search': StructuredTool(name='search', description='クエリを使ってWebを検索します', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x7f5799ea8b80>)}, tool_to_state_args={'search': {}}, tool_to_store_arg={'search': None}, handle_tool_errors=True)

In [63]:
model = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0).bind_tools(tools)

In [64]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"

    return END

In [65]:
# Test
should_continue({"messages": [AIMessage("こんにちは")]})

'__end__'

In [66]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

In [ ]:
# Test
call_model({"messages": [HumanMessage(content="こんにちは")]})

{'messages': [AIMessage(content='こんにちは。どのようなお手伝いができますでしょうか?', additional_kwargs={}, response_metadata={'id': 'msg_01FG9ioxFFADB93EE6264YeC', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 378, 'output_tokens': 26}}, id='run-6b28fcdd-3aa0-4f1d-aa49-17480f5bff00-0', usage_metadata={'input_tokens': 378, 'output_tokens': 26, 'total_tokens': 404, 'input_token_details': {}})]}

In [67]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

In [68]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="こんにちは")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'こんにちは。どのようなお手伝いができますでしょうか?'

In [69]:
final_state

{'messages': [HumanMessage(content='こんにちは', additional_kwargs={}, response_metadata={}, id='ea96068b-041b-43ec-9282-32ac894e6df4'),
  AIMessage(content='こんにちは。どのようなお手伝いができますでしょうか?', additional_kwargs={}, response_metadata={'id': 'msg_01WcBzPCWKZw5pCMwdqfwrUN', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 378, 'output_tokens': 26}}, id='run-3ac1d3c4-4221-425d-836d-0c00893ec16c-0', usage_metadata={'input_tokens': 378, 'output_tokens': 26, 'total_tokens': 404, 'input_token_details': {}})]}

In [70]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="東京の天気を教えて")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'東京の天気は現在、気温が20度前後で雨が降っているようです。傘があると良いでしょう。外出の際は雨対策をお忘れなく。'

In [71]:
final_state

{'messages': [HumanMessage(content='こんにちは', additional_kwargs={}, response_metadata={}, id='ea96068b-041b-43ec-9282-32ac894e6df4'),
  AIMessage(content='こんにちは。どのようなお手伝いができますでしょうか?', additional_kwargs={}, response_metadata={'id': 'msg_01WcBzPCWKZw5pCMwdqfwrUN', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 378, 'output_tokens': 26}}, id='run-3ac1d3c4-4221-425d-836d-0c00893ec16c-0', usage_metadata={'input_tokens': 378, 'output_tokens': 26, 'total_tokens': 404, 'input_token_details': {}}),
  HumanMessage(content='東京の天気を教えて', additional_kwargs={}, response_metadata={}, id='aaca72b8-b9ab-4fa4-b5c0-df254fef9f80'),
  AIMessage(content=[{'id': 'toolu_017rAeakjfu1p8SwpxzaAUyo', 'input': {'query': '東京の天気'}, 'name': 'search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_017pyN4zaTaF5rqfdKWGJhz5', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_token

In [74]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="北海道の天気を教えて")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'北海道の天気は現在、気温が25度前後で晴れ間が広がっているようです。外出するのに良い天気のようですね。日差しも強いので、日焼け対策をするのがよいでしょう。'

In [75]:
final_state

{'messages': [HumanMessage(content='こんにちは', additional_kwargs={}, response_metadata={}, id='ea96068b-041b-43ec-9282-32ac894e6df4'),
  AIMessage(content='こんにちは。どのようなお手伝いができますでしょうか?', additional_kwargs={}, response_metadata={'id': 'msg_01WcBzPCWKZw5pCMwdqfwrUN', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 378, 'output_tokens': 26}}, id='run-3ac1d3c4-4221-425d-836d-0c00893ec16c-0', usage_metadata={'input_tokens': 378, 'output_tokens': 26, 'total_tokens': 404, 'input_token_details': {}}),
  HumanMessage(content='東京の天気を教えて', additional_kwargs={}, response_metadata={}, id='aaca72b8-b9ab-4fa4-b5c0-df254fef9f80'),
  AIMessage(content=[{'id': 'toolu_017rAeakjfu1p8SwpxzaAUyo', 'input': {'query': '東京の天気'}, 'name': 'search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_017pyN4zaTaF5rqfdKWGJhz5', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_token

In [76]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="東京の天気を教えて")]},
    config={"configurable": {"thread_id": 10}}
)
final_state["messages"][-1].content

'東京の現在の天気は雨が降っており、気温は20度前後のようです。雨が降っているため、傘があると良いでしょう。外出の際は雨対策をしっかりと行うことをおすすめします。'

In [77]:
final_state

{'messages': [HumanMessage(content='東京の天気を教えて', additional_kwargs={}, response_metadata={}, id='04149953-126b-45f6-a3ef-00d3ddf8bdac'),
  AIMessage(content=[{'id': 'toolu_01Y3MMBAny346bhEmNsASFnj', 'input': {'query': '東京の天気'}, 'name': 'search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_016TQQJpLDiwRkEoUNRW1b3B', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 383, 'output_tokens': 55}}, id='run-8d3e1590-2549-409a-8b60-4c224f168ce9-0', tool_calls=[{'name': 'search', 'args': {'query': '東京の天気'}, 'id': 'toolu_01Y3MMBAny346bhEmNsASFnj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 383, 'output_tokens': 55, 'total_tokens': 438, 'input_token_details': {}}),
  ToolMessage(content='20度で雨が降っています。', name='search', id='2ad04070-85f1-4e8c-87e8-4723c0116ac5', tool_call_id='toolu_01Y3MMBAny346bhEmNsASFnj'),
  AIMessage(content='東京の現在の天気は雨が降っており、気温は20度前後のようです。雨が降っているため、傘があると良いでしょう。外出の際は雨対策をしっかりと行う